In [15]:
from whoosh.index import *
from whoosh.fields import *
from whoosh.qparser import *
from whoosh import scoring
from bs4 import BeautifulSoup as bs4 
import os
import json
INDEX_PATH = "./indexdir"
INDEX_NAME = "papers"

idx = open_dir(INDEX_PATH, indexname=INDEX_NAME)

from whoosh import qparser, query, scoring
from whoosh.analysis import RegexTokenizer
from whoosh.lang.morph_en import variations

freq_searcher = idx.searcher(weighting=scoring.Frequency())
tfidf_searcher = idx.searcher(weighting=scoring.TF_IDF())
bm25_searcher = idx.searcher(weighting=scoring.BM25F(B=0.74, K1=1.52))
query_parser = QueryParser('abstract', idx.schema)
query_parser.add_plugin(FuzzyTermPlugin())
title_parser = QueryParser('title', idx.schema)
title_parser.add_plugin(FuzzyTermPlugin())
tokenizer = RegexTokenizer()

In [40]:
def genearte_LETOR_data(qid, cur_query, docid, content, title, rel) :
    terms = cur_query.split(' ')
    q = query_parser.parse("path:\'"+docid+"\' "+" OR ".join(terms))
    q_2 = title_parser.parse("path:\'"+docid+"\' "+" OR ".join(terms))
    
    results = freq_searcher.search(q, limit=None)
    tf_a = 0.0
    if len(results):
#         print("Abstract TF feature", results[0].score)
        tf_a = results[0].score
    
    results = freq_searcher.search(q_2, limit=None)
    tf_t = 0.0
    if len(results):
#         print("Title TF feature", results[0].score)
        tf_t = results[0].score

    idf_a = sum(freq_searcher.idf("abstract", x) for x in terms)
#     print("Abstract IDF feature", idf_a)
    idf_t = sum(freq_searcher.idf("title", x) for x in terms)
#     print("Title IDF feature", idf_t)

    results = tfidf_searcher.search(q, limit=None)
    tfidf_a = 0.0
    if len(results):
#         print("Abstract TF-IDF feature", results[0].score)
        tfidf_a = results[0].score
    
    results = tfidf_searcher.search(q_2, limit=None)
    tfidf_t = 0.0
    if len(results):
#         print("Title TF-IDF feature", results[0].score)
        tfidf_t = results[0].score

    results = bm25_searcher.search(q, limit=None)
    bm25_a = 0.0
    if len(results):
#         print("Abstract BM25 feature", results[0].score)
        bm25_a = results[0].score
    
    results = bm25_searcher.search(q_2, limit=None)
    bm25_t = 0.0
    if len(results):
#         print("Title BM25 feature", results[0].score)
        bm25_t = results[0].score

    dl = len(list(x for x in tokenizer(content)))
#     print("DL feature", dl)
    
    tl = len(list(x for x in tokenizer(title)))
#     print("TL feature", tl)
    
    return rel + " qid:%s 1:%f 2:%f 3:%f 4:%f 5:%f 6:%f 7:%f 8:%f 9:%f 10:%f #docid = %s\n" % (qid, tf_a, tf_t, idf_a, idf_t, tfidf_a, tfidf_t, bm25_a, bm25_t, dl, tl, docid)

In [34]:
# Build the query map
query_map = dict()
with open("queries.indri", 'r') as q:
    soup = bs4(q, 'lxml')
    x = soup.find_all('query')
    i = 1
    for query in x :
        query_1 = query.text.strip().split('\n')
        query_map[str(i)] = query_1[1]
        i += 1

with open("./bob_acl_anth.qrels", 'r') as qrels, open('train_f.txt', 'w+') as w :
    for line in qrels :
        data = line.strip().split('\t')
        qid = data[0]
        docid = data[2]
        rel = data[3]
        cur_query = query_map[qid]
        stored = freq_searcher.document(path=docid+".tei.xml")
        if not stored :
            continue
        w.write(genearte_LETOR_data(qid, cur_query, docid+".tei.xml", stored['abstract'], stored['title'], rel))           

Abstract IDF feature 92.4560724513172
Title IDF feature 104.56255732141997
DL feature 91
TL feature 6
Abstract IDF feature 92.4560724513172
Title IDF feature 104.56255732141997
DL feature 0
TL feature 7
Abstract IDF feature 92.4560724513172
Title IDF feature 104.56255732141997
DL feature 140
TL feature 5
Abstract IDF feature 92.4560724513172
Title IDF feature 104.56255732141997
DL feature 22
TL feature 5
Abstract IDF feature 92.4560724513172
Title IDF feature 104.56255732141997
DL feature 137
TL feature 6
Abstract IDF feature 92.4560724513172
Title IDF feature 104.56255732141997
DL feature 78
TL feature 6
Abstract IDF feature 92.4560724513172
Title IDF feature 104.56255732141997
DL feature 311
TL feature 7
Abstract IDF feature 92.4560724513172
Title IDF feature 104.56255732141997
DL feature 62
TL feature 14
Abstract IDF feature 92.4560724513172
Title IDF feature 104.56255732141997
DL feature 69
TL feature 8
Abstract IDF feature 92.4560724513172
Title IDF feature 104.56255732141997
DL f

Abstract TF feature 11.0
Title TF feature 3.0
Abstract IDF feature 64.49623960490962
Title IDF feature 67.12048582819972
Abstract TF-IDF feature 49.23167738548157
Title TF-IDF feature 20.76465190152212
Abstract BM25 feature 16.12837739875028
Title BM25 feature 11.778901804717318
DL feature 257
TL feature 10
Abstract TF feature 10.0
Title TF feature 6.0
Abstract IDF feature 64.49623960490962
Title IDF feature 67.12048582819972
Abstract TF-IDF feature 43.08131738412866
Title TF-IDF feature 33.8854671825644
Abstract BM25 feature 21.330901394283728
Title BM25 feature 19.245745786010808
DL feature 67
TL feature 11
Abstract IDF feature 31.632573305569885
Title IDF feature 36.105225673384595
DL feature 514
TL feature 7
Abstract IDF feature 31.632573305569885
Title IDF feature 36.105225673384595
DL feature 69
TL feature 10
Title TF feature 2.0
Abstract IDF feature 31.632573305569885
Title IDF feature 36.105225673384595
Title TF-IDF feature 16.112882136766654
Title BM25 feature 7.05186404942154

Abstract IDF feature 162.7096826022062
Title IDF feature 197.45717373512363
DL feature 154
TL feature 16
Abstract IDF feature 162.7096826022062
Title IDF feature 197.45717373512363
DL feature 106
TL feature 10
Abstract IDF feature 162.7096826022062
Title IDF feature 197.45717373512363
DL feature 88
TL feature 64
Abstract IDF feature 162.7096826022062
Title IDF feature 197.45717373512363
DL feature 90
TL feature 4
Abstract IDF feature 98.99568033369104
Title IDF feature 122.53810893846364
DL feature 0
TL feature 123
Abstract TF feature 3.0
Abstract IDF feature 98.99568033369104
Title IDF feature 122.53810893846364
Abstract TF-IDF feature 18.36827479620158
Abstract BM25 feature 9.317064266321873
DL feature 104
TL feature 11
Abstract TF feature 3.0
Abstract IDF feature 98.99568033369104
Title IDF feature 122.53810893846364
Abstract TF-IDF feature 20.327247200758816
Abstract BM25 feature 9.216322138322935
DL feature 56
TL feature 10
Abstract IDF feature 98.99568033369104
Title IDF feature 

Abstract BM25 feature 10.079051933435434
Title BM25 feature 18.939974564733543
DL feature 153
TL feature 8
Abstract TF feature 17.0
Title TF feature 5.0
Abstract IDF feature 108.89390630210647
Title IDF feature 134.44464349482573
Abstract TF-IDF feature 61.883922155988905
Title TF-IDF feature 28.11012853918465
Abstract BM25 feature 23.764098893078902
Title BM25 feature 20.91279699275468
DL feature 186
TL feature 8
Abstract TF feature 7.0
Title TF feature 3.0
Abstract IDF feature 108.89390630210647
Title IDF feature 134.44464349482573
Abstract TF-IDF feature 35.904017014406314
Title TF-IDF feature 19.746404046193902
Abstract BM25 feature 24.53563155888299
Title BM25 feature 10.15069629550727
DL feature 68
TL feature 9
Abstract TF feature 10.0
Title TF feature 3.0
Abstract IDF feature 108.89390630210647
Title IDF feature 134.44464349482573
Abstract TF-IDF feature 45.295692252223205
Title TF-IDF feature 21.87657305648589
Abstract BM25 feature 36.032390578261726
Title BM25 feature 12.99077

Title BM25 feature 9.390555456524384
DL feature 56
TL feature 11
Abstract TF feature 10.0
Title TF feature 5.0
Abstract IDF feature 90.34485415969442
Title IDF feature 107.20541401357636
Abstract TF-IDF feature 39.13371056856936
Title TF-IDF feature 28.33558725824617
Abstract BM25 feature 21.018376428335962
Title BM25 feature 20.030367541168914
DL feature 85
TL feature 9
Abstract TF feature 5.0
Title TF feature 2.0
Abstract IDF feature 90.34485415969442
Title IDF feature 107.20541401357636
Abstract TF-IDF feature 26.14337943421647
Title TF-IDF feature 14.482977737191236
Abstract BM25 feature 16.095905530955193
Title BM25 feature 4.9325853732218965
DL feature 87
TL feature 7
Abstract TF feature 2.0
Abstract IDF feature 156.6197711422267
Title IDF feature 186.70464966430882
Abstract TF-IDF feature 16.123634928542916
Abstract BM25 feature 7.3445641424467345
DL feature 69
TL feature 10
Abstract TF feature 2.0
Title TF feature 2.0
Abstract IDF feature 156.6197711422267
Title IDF feature 186

...

Abstract TF feature 5.0
Title TF feature 3.0
Abstract IDF feature 114.28481978018587
Title IDF feature 134.6813331384893
Abstract TF-IDF feature 25.14893541776137
Title TF-IDF feature 19.399554934214272
Abstract BM25 feature 12.344133293199942
Title BM25 feature 10.291100057384913
DL feature 73
TL feature 8
Abstract TF feature 5.0
Title TF feature 3.0
Abstract IDF feature 114.28481978018587
Title IDF feature 134.6813331384893
Abstract TF-IDF feature 26.87019428397366
Title TF-IDF feature 19.399554934214272
Abstract BM25 feature 18.48717401784617
Title BM25 feature 10.849097991613213
DL feature 68
TL feature 8
Abstract TF feature 14.0
Title TF feature 5.0
Abstract IDF feature 114.28481978018587
Title IDF feature 134.6813331384893
Abstract TF-IDF feature 63.12573478545024
Title TF-IDF feature 32.19572630287752
Abstract BM25 feature 29.8526438584105
Title BM25 feature 25.63306318855149
DL feature 103
TL feature 7
Abstract TF feature 2.0
Abstract IDF feature 114.59966757996159
Title IDF fe

Abstract BM25 feature 4.767978584484824
Title BM25 feature 5.937155509922722
DL feature 124
TL feature 9
Abstract IDF feature 135.25336395195035
Title IDF feature 164.3547914423473
DL feature 77
TL feature 12
Abstract IDF feature 135.25336395195035
Title IDF feature 164.3547914423473
DL feature 138
TL feature 12
Title TF feature 4.0
Abstract IDF feature 135.25336395195035
Title IDF feature 164.3547914423473
Title TF-IDF feature 28.200868447505414
Title BM25 feature 18.871059288009118
DL feature 0
TL feature 9
Abstract TF feature 9.0
Title TF feature 4.0
Abstract IDF feature 135.25336395195035
Title IDF feature 164.3547914423473
Abstract TF-IDF feature 43.05306656090979
Title TF-IDF feature 28.465067387851924
Abstract BM25 feature 19.65942683538374
Title BM25 feature 17.386387472397796
DL feature 169
TL feature 12
Abstract TF feature 6.0
Title TF feature 2.0
Abstract IDF feature 135.25336395195035
Title IDF feature 164.3547914423473
Abstract TF-IDF feature 32.93382406105834
Title TF-IDF

Abstract TF feature 3.0
Abstract IDF feature 91.1204604127542
Title IDF feature 112.54840297426341
Abstract TF-IDF feature 15.36196929616329
Abstract BM25 feature 4.564573919990446
DL feature 192
TL feature 13
Abstract TF feature 4.0
Title TF feature 2.0
Abstract IDF feature 91.1204604127542
Title IDF feature 112.54840297426341
Abstract TF-IDF feature 18.49599832598976
Title TF-IDF feature 16.07613259455791
Abstract BM25 feature 8.000152691647328
Title BM25 feature 6.668944501786797
DL feature 70
TL feature 10
Abstract TF feature 5.0
Title TF feature 2.0
Abstract IDF feature 91.1204604127542
Title IDF feature 112.54840297426341
Abstract TF-IDF feature 24.08127549877561
Title TF-IDF feature 18.01178624344604
Abstract BM25 feature 13.661159147198626
Title BM25 feature 6.80522734842228
DL feature 144
TL feature 23
Abstract TF feature 2.0
Abstract IDF feature 91.1204604127542
Title IDF feature 112.54840297426341
Abstract TF-IDF feature 16.111101192395658
Abstract BM25 feature 5.82536427880

Title TF-IDF feature 31.068583924504665
Abstract BM25 feature 42.45468020569848
Title BM25 feature 21.828966345346743
DL feature 69
TL feature 10
Abstract TF feature 23.0
Title TF feature 6.0
Abstract IDF feature 262.85467771515056
Title IDF feature 295.50712187979116
Abstract TF-IDF feature 95.88829570998944
Title TF-IDF feature 37.70343820741362
Abstract BM25 feature 39.89560329763923
Title BM25 feature 28.672491257503957
DL feature 112
TL feature 11
Abstract TF feature 5.0
Title TF feature 4.0
Abstract IDF feature 262.85467771515056
Title IDF feature 295.50712187979116
Abstract TF-IDF feature 23.957091009734278
Title TF-IDF feature 24.192056695309155
Abstract BM25 feature 14.358633697828832
Title BM25 feature 16.38608067190173
DL feature 81
TL feature 9
Abstract TF feature 5.0
Abstract IDF feature 262.85467771515056
Title IDF feature 295.50712187979116
Abstract TF-IDF feature 23.442549227202285
Abstract BM25 feature 11.591624256476077
DL feature 49
TL feature 8
Abstract TF feature 1

Title BM25 feature 12.418257432613931
DL feature 112
TL feature 11
Abstract TF feature 3.0
Abstract IDF feature 312.42447480703026
Title IDF feature 373.65228439287216
Abstract TF-IDF feature 16.317080238361193
Abstract BM25 feature 7.578389237221787
DL feature 81
TL feature 9
Abstract TF feature 9.0
Abstract IDF feature 312.42447480703026
Title IDF feature 373.65228439287216
Abstract TF-IDF feature 49.15918245713751
Abstract BM25 feature 27.221588833579666
DL feature 75
TL feature 13
Abstract IDF feature 312.42447480703026
Title IDF feature 373.65228439287216
DL feature 100
TL feature 10
Abstract TF feature 8.0
Title TF feature 4.0
Abstract IDF feature 312.42447480703026
Title IDF feature 373.65228439287216
Abstract TF-IDF feature 46.736478889191645
Title TF-IDF feature 28.942187117995836
Abstract BM25 feature 23.4555517212766
Title BM25 feature 24.723666326080977
DL feature 101
TL feature 6
Title TF feature 3.0
Abstract IDF feature 312.42447480703026
Title IDF feature 373.65228439287

In [35]:
# Train data using Learning-To-Rank LambdaMART model
import pyltr

with open('train_f.txt') as f :
    features, rels, qids, comment = pyltr.data.letor.read_dataset(f)
    metric = pyltr.metrics.NDCG(k=10)
    model = pyltr.models.LambdaMART(
        metric=metric,
        n_estimators=800,
        learning_rate=0.02,
        max_features=0.5,
        query_subsample=0.5,
        max_leaf_nodes=10,
        min_samples_leaf=64,
        verbose=1,
    )
    model.fit(features, rels, qids)

 Iter  Train score  OOB Improve    Remaining                           Monitor Output 
    1       0.3877       0.2878       41.86s                                         
    2       0.4648       0.0531       42.88s                                         
    3       0.4629       0.0260       45.65s                                         
    4       0.5201       0.0054       48.15s                                         
    5       0.5392       0.0028       50.62s                                         
    6       0.4770       0.0067       53.02s                                         
    7       0.5034       0.0044        1.11m                                         
    8       0.4921       0.0019        1.11m                                         
    9       0.5212       0.0027        1.13m                                         
   10       0.5498      -0.0020        1.28m                                         
   15       0.5400       0.0000        1.29m         

In [36]:
# serialize the model
import pickle

with open('ltr_model.pickle', 'wb') as f:
    # Pickle the 'ltr_model' dictionary using the highest protocol available.
    pickle.dump(model, f, pickle.HIGHEST_PROTOCOL)

In [37]:
with open('ltr_model.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    model = pickle.load(f)

In [50]:
from whoosh import qparser, query, scoring

words = "information"
with open('result_f.txt.tmp', 'w+') as wr :
    searcher = idx.searcher(weighting=scoring.BM25F(B=0.74, K1=1.52))
    query_parser = MultifieldParser(['title', 'abstract', 'area'], idx.schema, termclass=query.Variations)
    query_parser.add_plugin(FuzzyTermPlugin())
    query_parsed = query.And([query.Variations('abstract', x) for x in words.split(' ') if len(x) > 1])
    results = searcher.search(query_parsed, limit=100)
    for r in results :
        wr.write(genearte_LETOR_data('1', words, r['path'], r['abstract'], r['title'], '1'))

with open('result_f.txt.tmp', 'r') as f :
    features, _, qids, docs = pyltr.data.letor.read_dataset(f)
    p = model.predict(features)
    result = []
    for j in range(len(qids)) :
        result.append((p[j], searcher.document(path=docs[j][8:].strip())))
    result.sort(key = lambda x:x[0], reverse = True)
result

[(0.7924560110721995,
  {'abstract': 'We present a framework for the integrated analysis of the textual and prosodic characteristics of information structure in the Switchboard corpus of conversational En-glish. Information structure describes the availability, organisation and salience of entities in a discourse model. We present standards for the annotation of information status (old, mediated and new), and give guidelines for annotating information structure, i.e. theme/rheme and back-ground/kontrast. We show that information structure in English can only be analysed concurrently with prosodic prominence and phrasing. This annotation, using stand-off XML in NXT, can help establish standards for the annotation of information structure in discourse.',
   'area': '',
   'path': 'W05-0307.tei.xml',
   'title': 'A Framework for Annotating Information Structure in Discourse'}),
 (0.7603878087648669,
  {'abstract': 'This paper provides a quick summary of the following topics: enhancements 

In [59]:
docnum = searcher.document_number(path=u"W05-0307.tei.xml")
r = searcher.more_like(docnum, 'abstract', top = 30)

<Hit {'abstract': 'We demonstrate work in progress 1 using the Nite XML Toolkit on a corpus of multimodal dialogues with an MP3 player collected in a Wizard-of-Oz (WOZ) experiments and annotated with a rich feature set at several layers. We designed an NXT data model, converted experiment log file data and manual transcriptions into NXT, and are building annotation tools using NXT libraries.', 'area': '', 'path': 'W06-2711.tei.xml', 'title': 'The SAMMIE Multimodal Dialogue Corpus Meets the Nite XML Toolkit'}>
<Hit {'abstract': 'It is natural to expect phrase structure to be important in predicting prosodic phrasing. Yet there appears to be a concensus that syntactic phrases do not correspond well to prosodic phrasing, and independent structures have been proposed to account for prosody. I propose that the problem with phrase structure lies with the particular measures of boundary strength applied to syntactic structures, and with the fact that phrase structure is viewed as an immediate